In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [3]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)]
  wgt = [(0.7, 0.3, 0.7), (0.7, 0.6), (0.4, 0.5, 0.8, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.387582                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.241821
0.000750      2.0  0.387546                  2.0  0.246630
0.001052      3.0  0.400928                  3.0  0.258910
0.001476      4.0  0.412757                  4.0  0.271161
0.002070      5.0  0.422496                  5.0  0.283025
0.002904      6.0  0.428964                  6.0  0.294171
0.004074      7.0  0.431441                  7.0  0.304121
0.005714      8.0  0.428594                  8.0  0.312299
0.008015      9.0  0.419080                  9.0  0.317726
0.011243     10.0  0.401382                 10.0  0.319275
0.015771     11.0  0.373889                 11.0  0.315202
0.022122     12.0  0.334802                 12.0  0.302964
0.031031     13.0  0.282050                 13.0  0.279572
0.043528     14.0  0.217212                 14.0  0.245146
0.061057     15.0  0.193478                 15.0  0.241421
0.085645     16.0  0.237149                 16.0  0.289261
0.120136     17.0  0.294388                 17.0  0.339616
0.168516     18.0  0.335502                 18.0  0.357516
0.236378     19.0  0.370358                 19.0  0.350804
0.331549     20.0  0.415121                 20.0  0.353648
0.465100     21.0  0.494583                 21.0  0.414042
0.652400     22.0  0.622851                 22.0  0.550407
0.915100     23.0  0.624268                 23.0  0.587195
1.283650     24.0  0.524220                 24.0  0.518556
1.800600     25.0  0.441412                 25.0  0.443063
2.525700     26.0  0.355842                 26.0  0.360517
3.542800     27.0  0.288916                 27.0  0.304139
4.969550     28.0  0.236564                 28.0  0.263970
6.970850     29.0  0.192368                 29.0  0.227440
9.778100     30.0  0.154051                 30.0  0.184614
13.715850    31.0  0.120743                 31.0  0.140424
19.239350    32.0  0.092200                 32.0  0.100345
26.987250    33.0  0.068633                 33.0  0.068801
37.855300    34.0  0.050726                 34.0  0.049357
53.100050    35.0  0.025755                 35.0  0.028054
73.887500    36.0 -0.003896                 36.0 -0.000163
97.662500    37.0 -0.022584                 37.0 -0.020594
121.437500   38.0 -0.019633                 38.0 -0.017627
145.212500   39.0 -0.003142                 39.0 -0.000269
168.987500   40.0  0.007541                 40.0  0.010792
192.762500   41.0  0.017182                 41.0  0.020773
216.537500   42.0  0.026291                 42.0  0.029534
240.312500   43.0  0.034933                 43.0  0.036952
264.087500   44.0  0.043120                 44.0  0.043258
287.862500   45.0  0.050840                 45.0  0.048882
311.637500   46.0  0.058081                 46.0  0.054032
335.412500   47.0  0.064840                 47.0  0.058960
359.187500   48.0  0.071120                 48.0  0.063926
382.962500   49.0  0.076922                 49.0  0.068953
406.737500   50.0  0.082269                 50.0  0.074073
430.512500   51.0  0.087213                 51.0  0.079349
454.287500   52.0  0.091786                 52.0  0.084772
478.062500   53.0  0.096022                 53.0  0.090218
501.837500   54.0  0.099969                 54.0  0.095626
525.612500   55.0  0.103684                 55.0  0.100821
549.387500   56.0  0.107224                 56.0  0.105927
573.162500   57.0  0.110673                 57.0  0.110862
596.937500   58.0  0.114144                 58.0  0.115645
620.712500   59.0  0.117747                 59.0  0.120345
644.487500   60.0  0.121624                 60.0  0.125046
668.262500   61.0  0.125609                 61.0  0.129615
692.037500   62.0  0.128635                 62.0  0.133100
715.812500   63.0  0.128751                 63.0  0.133675
739.587500   64.0  0.125153                 64.0  0.130669
76

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -30.870337   0.000000 -30.870337        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -30.990152   0.000002   
6.244000e-04 2     -30.870355   0.000046 -30.870308 -30.990169   0.000037   
8.759000e-04 3     -30.870362   0.000065 -30.870297 -30.990176   0.000051   
1.228600e-03 4     -30.870372   0.000092 -30.870280 -30.990186   0.000071   
1.723400e-03 5     -30.870385   0.000130 -30.870256 -30.990199   0.000100   
2.417400e-03 6     -30.870404   0.000183 -30.870221 -30.990217   0.000142   
3.390900e-03 7     -30.870430   0.000259 -30.870172 -30.990243   0.000201   
4.756500e-03 8     -30.870466   0.000365 -30.870102 -30.990278   0.000286   
6.672000e-03 9     -30.870516   0.000512 -30.870005 -30.990327   0.000406   
9.358900e-03 10    -30.870586   0.000715 -30.869871 -30.990396   0.000575   
1.312780e-02 11    -30.870684   0.000992 -30.869692 -30.990492   0.000814   
1.841450e-02 12    -30.870822   0.001364 -30.869458 -30.990628   0.001148   
2.583020e-02 13    -30.871020   0.001856 -30.869164 -30.990823   0.001609   
3.623230e-02 14    -30.871310   0.002494 -30.868816 -30.991105   0.002236   
5.082340e-02 15    -30.871748   0.003307 -30.868441 -30.991521   0.003075   
7.129060e-02 16    -30.872394   0.004423 -30.867972 -30.992119   0.004258   
1.000000e-01 17    -30.873284   0.006119 -30.867165 -30.992923   0.006047   
1.402710e-01 18    -30.874444   0.008683 -30.865761 -30.993949   0.008693   
1.967600e-01 19    -30.875906   0.012390 -30.863516 -30.995205   0.012341   
2.759970e-01 20    -30.877654   0.017614 -30.860039 -30.996655   0.017084   
3.871000e-01 21    -30.879535   0.024959 -30.854576 -30.998139   0.023223   
5.431000e-01 22    -30.880927   0.035491 -30.845437 -30.999095   0.031831   
7.617000e-01 23    -30.879841   0.050533 -30.829308 -30.997724   0.044714   
1.068500e+00 24    -30.875457   0.068836 -30.806620 -30.993285   0.061617   
1.498800e+00 25    -30.868780   0.088880 -30.779900 -30.986577   0.081344   
2.102400e+00 26    -30.859879   0.111540 -30.748339 -30.977560   0.104010   
2.949000e+00 27    -30.849519   0.136866 -30.712653 -30.966796   0.129404   
4.136600e+00 28    -30.837908   0.165900 -30.672008 -30.954230   0.159629   
5.802500e+00 29    -30.824962   0.199637 -30.625325 -30.939572   0.197068   
8.139200e+00 30    -30.810726   0.238648 -30.572078 -30.922511   0.242968   
1.141700e+01 31    -30.795474   0.283211 -30.512263 -30.903151   0.295298   
1.601470e+01 32    -30.779838   0.333336 -30.446503 -30.882031   0.350665   
2.246400e+01 33    -30.764971   0.388906 -30.376065 -30.860539   0.405841   
3.151050e+01 34    -30.752427   0.449910 -30.302517 -30.840601   0.459639   
4.420010e+01 35    -30.742088   0.515821 -30.226267 -30.821939   0.515176   
6.200000e+01 36    -30.746706   0.574745 -30.171962 -30.819311   0.571705   
8.577500e+01 37    -30.798906   0.615973 -30.182933 -30.872434   0.624369   
1.095500e+02 38    -30.900056   0.653519 -30.246537 -30.984289   0.678219   
1.333250e+02 39    -31.038851   0.737021 -30.301830 -31.137625   0.781908   
1.571000e+02 40    -31.196946   0.886269 -30.310677 -31.308711   0.952234   
1.808750e+02 41    -31.371584   1.082145 -30.289439 -31.493903   1.167822   
2.046500e+02 42    -31.560530   1.319481 -30.241048 -31.691578   1.424006   
2.284250e+02 43    -31.761822   1.594818 -30.167004 -31.901138   1.716753   
2.522000e+02 44    -31.973608   1.904987 -30.068621 -32.120392   2.040087   
2.759750e+02 45    -32.194332   2.247150 -29.947182 -32.347440   2.388976   
2.997500e+02 46    -32.422620   2.618619 -29.804000 -32.580806   2.760024   
3.235250e+02 47    -32.657242   3.016817 -29.640426 -32.819164   3.150569   
3.473000e+02 48    -32.897095   3.439278 -29.457817 -33.061188   3.558661   
3.710750e+02 49    -33

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')